In [12]:
import os

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

from scipy.stats import poisson

import matplotlib.pyplot as plt

In [13]:
C=70.0
downweight=0.8
risk_spill=0.95
risk_spoil=risk_spill
downweight_granularity=['dtime','Direction','dday','month']

In [14]:
C_datadriven=pd.read_csv('~/Data/Intermediate_Output/R_Training_Pax.csv')
C_datadriven=C_datadriven.groupby(['DepDate']+downweight_granularity)['NumPax'].sum().reset_index()
C_datadriven=C_datadriven.groupby(downweight_granularity)['NumPax'].max().reset_index().rename(columns={'NumPax': 'downweight'})
C_datadriven['downweight']=C_datadriven['downweight'].astype('float')/C
C_datadriven['downweight']=C_datadriven['downweight'].apply(lambda x: downweight if x>downweight else x)
# C_datadriven['downweight']=downweight

In [15]:
Prediction=pd.read_csv('~/Data/Intermediate_Output/R_Output_Training_Pax.csv',sep=',')

In [16]:
Prediction=Prediction.merge(C_datadriven,on=downweight_granularity)

IdealCurve=Prediction[['Dprio']+downweight_granularity+['forecast_bookings','forecast_node','downweight']]\
.drop_duplicates(subset=['Dprio']+downweight_granularity)

IdealCurve=IdealCurve.merge(IdealCurve.groupby(downweight_granularity)['forecast_bookings'].sum().reset_index()\
.rename(columns={'forecast_bookings': 'forecast_bookings_sum'}),on=downweight_granularity)

In [17]:
IdealCurve['forecast_bookings_full']=C*(IdealCurve['forecast_bookings']/IdealCurve['forecast_bookings_sum'])
IdealCurve['forecast_bookings_downweighted']=C*IdealCurve['downweight']*(IdealCurve['forecast_bookings']/IdealCurve['forecast_bookings_sum'])

In [18]:
IdealCurve['Cap_full']=C
IdealCurve['Cap_downweighted']=C*IdealCurve['downweight']

In [19]:
Probability=IdealCurve[['Dprio','dtime','Direction','dday','month','Cap_full','Cap_downweighted']].copy()

In [20]:
IdealCurve.to_csv('~/Data/Intermediate_Output/IdealCurve_forYield.csv',index=False)

# Cumulation

In [21]:
IdealCurve=IdealCurve[[x for x in IdealCurve.columns if 'sum' not in x and x!='forecast_bookings']]

In [22]:
IdealCurve['Dprio']=-IdealCurve['Dprio']
IdealCurve=IdealCurve.set_index(['Dprio']+downweight_granularity+['downweight'])

IdealCurve_average_full=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['forecast_bookings_full'].cumsum().reset_index().rename(columns={'forecast_bookings_full': 'Ideal_average_full'})
IdealCurve_average_downweighted=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['forecast_bookings_downweighted'].cumsum().reset_index().rename(columns={'forecast_bookings_downweighted': 'Ideal_average_downweighted'})

IdealCurve_average_full['Dprio']=-IdealCurve_average_full['Dprio']
IdealCurve_average_downweighted['Dprio']=-IdealCurve_average_downweighted['Dprio']

In [23]:
IdealCurve=IdealCurve.reset_index()
IdealCurve['Dprio']=-IdealCurve['Dprio']
IdealCurve=IdealCurve.set_index(['Dprio']+downweight_granularity+['downweight'])
IdealCurve_intermediate_full=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['forecast_bookings_full'].cumsum().reset_index().rename(columns={'forecast_bookings_full': 'Ideal_intermediate_full'})
IdealCurve_intermediate_downweighted=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['forecast_bookings_downweighted'].cumsum().reset_index().rename(columns={'forecast_bookings_downweighted': 'Ideal_intermediate_downweighted'})

In [24]:
Probability=Probability.merge(IdealCurve_intermediate_full[[x for x in IdealCurve_intermediate_full if x!='downweight']],
                              on=['Dprio']+downweight_granularity)

Probability=Probability.merge(IdealCurve_intermediate_downweighted[[x for x in IdealCurve_intermediate_downweighted if x!='downweight']],
                              on=['Dprio']+downweight_granularity)

Probability=Probability.merge(IdealCurve_average_full[[x for x in IdealCurve_average_full if x!='downweight']],
                              on=['Dprio']+downweight_granularity)

Probability=Probability.merge(IdealCurve_average_downweighted[[x for x in IdealCurve_average_downweighted if x!='downweight']],
                              on=['Dprio']+downweight_granularity)

In [25]:
Probability.to_csv('~/Data/Intermediate_Output/Probability.csv',index=False)

In [26]:
IdealCurve=IdealCurve_average_full.merge(IdealCurve_average_downweighted,on=['Dprio']+downweight_granularity+['downweight'])
IdealCurve=IdealCurve.merge(IdealCurve_intermediate_full,on=['Dprio']+downweight_granularity+['downweight'])
IdealCurve=IdealCurve.merge(IdealCurve_intermediate_downweighted,on=['Dprio']+downweight_granularity+['downweight'])

In [27]:
IdealCurve['Ideal_lower']=C*IdealCurve['downweight']-poisson.ppf(risk_spoil,IdealCurve['Ideal_intermediate_downweighted'])
IdealCurve['Ideal_lower']=IdealCurve['Ideal_lower'].apply(lambda x: 0 if x<0 else x)

IdealCurve['Ideal_upper']=C-poisson.ppf(1-risk_spill,IdealCurve['Ideal_intermediate_full'])
IdealCurve['Ideal_upper']=IdealCurve['Ideal_upper'].apply(lambda x: C if x>C else x)

In [28]:
IdealCurve=IdealCurve[downweight_granularity+['Dprio','Ideal_lower','Ideal_average_downweighted','Ideal_average_full','Ideal_upper']]
IdealCurve.to_csv('~/Data/Intermediate_Output/IdealCurve.csv',index=False)